# Modify Spacy Lemmas

  * Spacy's lemmas are stored in a binary file as part of the model since version 2.2
  * 
  
  
conda install spacy-lookups-data



Find the location of the lemma file:

In [4]:
import spacy_lookups_data

fname = spacy_lookups_data.de['lemma_lookup']
print(f"Lemma file: {fname}")

Lemma file: C:\Users\Jens\Anaconda3\lib\site-packages\spacy_lookups_data\data\de_lemma_lookup.json


In [ ]:
import spacy_lookups_data

spacy_lookups_data.de['lemma_lookup']

In [ ]:
?nlp.vocab.lookups.remove_table

In [ ]:
import json
import spacy_lookups_data

fname = spacy_lookups_data.de['lemma_lookup']

with open(fname, 'r') as file:
    lemma_lookup = json.load(file)

lemma_lookup['Sonne']

In [ ]:
nlp.vocab.lookups.add_table('lemma_lookup', lemma_lookup)

In [ ]:
nlp.vocab.lookups.tables

In [3]:
import spacy
import spacy_lookups_data
import json

fname = spacy_lookups_data.de['lemma_lookup']

with open(fname, 'r') as file:
    lemma_lookup = json.load(file)

nlp = spacy.load('de')

# must be executed before first call to nlp
nlp.vocab.lookups.remove_table('lemma_lookup')
nlp.vocab.lookups.add_table('lemma_lookup', lemma_lookup)

texts = ["Ich sonne mich in der Sonne.", 
         "Dieser Gärtner wohnt im Haus."]

for text in texts:
    doc = nlp(text)
    print(" ".join([t.lemma_ for t in doc]))

Lemma file: C:\Users\Jens\Anaconda3\lib\site-packages\spacy_lookups_data\data\de_lemma_lookup.json


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Jens\\Anaconda3\\lib\\site-packages\\spacy_lookups_data\\data\\de_lemma_lookup.json'

In [ ]:
nlp.vocab.lookups.

In [ ]:
C:\Users\Jens\Anaconda3\lib\site-packages\spacy_lookups_data\data

# Nouns from Wiktionary

https://github.com/gambolputty/german-nouns

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

# IPython (Jupyter) setting: 
# Print out every value instead of just "last_expr" (default)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df = pd.read_csv('nouns.csv', low_memory=False)
df.sample(10)

In [ ]:
df['nominativ plural']

In [ ]:
df[df['lemma'] == 'Verkauf']

In [ ]:
# for i, col in enumerate(df.columns):
#    print(i, col)
    
cols = df.columns[16:]
cols

In [ ]:
df[cols] = df[cols].fillna('')
df = df[df['pos'] == 'Substantiv']

In [ ]:
import re

nouns = {}

for i, row in df.iterrows():
    lemma = row['lemma']
    
    # skip non-nouns, tokens with specials and acronyms
    if re.search(r'[,\-\s\.0-9]', lemma)!=None or re.search(r'[A-Z]', lemma[1:])!=None:
        continue
        
    nouns[lemma] = lemma

    # add flex forms
    for col in cols:
        flex = row[col]

        if flex != None and len(flex) > 1 and \
           re.search(r'[,\-\s\.0-9]', flex)==None and re.search(r'[A-Z]', flex[1:])==None:
            nouns[flex] = lemma
            
nouns

In [ ]:
nouns['Haus']

In [ ]:
# df = orig_df.copy()

In [ ]:
df[df.lemma == 'All']

In [ ]:
df.pos.value_counts()

# Correction of spaCy's German Lemmas v.2.1

In [ ]:
nlp.vocab.lookups.get_table('lemma_lookup')

In [ ]:
nlp.vocab[16219792130206718131]

In [ ]:
from spacy.lang.de.lemmatizer import LOOKUP
import sys

In [ ]:
# file = sys.stdout
file = open("lemmatizer_de.py", "w", encoding='utf-8')
# file = open("lemmatizer_de_changes.py", "w", encoding='utf-8')

file.write("""# coding: utf8
from __future__ import unicode_literals

LOOKUP_DELTA = {\n""")

for word, lemma in LOOKUP.items():
    try:
        if word[0].isupper() and lemma.islower() and word.lower() not in LOOKUP and word in nouns:
            new_lemma = nouns[word]
            file.write(f'    "{word}": "{new_lemma}",\n')
            # file.write(f'    "{word}": "{new_lemma}", # previous "{lemma}"\n')
            # print(f'    "{word}": "{new_lemma}" # previous "{lemma}"')
        else:
            # pass
            file.write(f'    "{word}": "{lemma}",\n')
        
    except Exception as e:
        print(e)
        print(f'    "{word}": "{lemma}"')
        break
            
file.write("}\n")
file.close()

In [ ]:
word = "Banden"
word[-2:]

In [ ]:
"Banden"[:-1] in LOOKUP

In [ ]:
"Bande" in LOOKUP

# Test

In [ ]:
import spacy

nlp = spacy.load('de')

In [ ]:
from IPython.core.display import display, HTML
from tabulate import tabulate

def print_nlp(doc, include_punct=False):
    """Print tokens with attributes for spaCy doc."""
    rows = []
    for token in doc:
        if not token.is_punct or include_punct:
            row = (token.text, token.lemma_, 
                   token.pos_, token.tag_, token.dep_,
                   token.is_punct, token.is_alpha, token.is_stop,
                   token.ent_type_, token.ent_iob_)
            rows.append(row)

    # generate HTML formatted table for display in Jupyter
    headers = ['text', 'lemma_', 'pos_', 'tag_', 'dep_', 
               'is_punct', 'is_alpha', 'is_stop', 'ent_type', 'ent_iob'] 
    display(HTML(tabulate(rows, headers=headers, tablefmt='html')))
    

In [ ]:
from lemmatizer_de import LOOKUP as my_LOOKUP

nlp.vocab.morphology.lemmatizer.lookup_table = my_LOOKUP




In [ ]:
LOOKUP['Baum']

In [ ]:
import spacy

nlp = spacy.blank('de')
#nlp = spacy.load('de')

def print_nlp(doc):
    print(" ".join([f"{t}/{t.lemma_}/{t.pos_}" for t in doc if not t.is_punct]))

texts = """Dieser Gärtner wohnt im Haus.
"""

for text in texts.split("\n"):
    doc = nlp(text)
    print_nlp(doc)

In [ ]:
spacy.lookups

# SpaCy 2.2

In [ ]:
import sys

sys.path.insert(0, r'/home/albrecht/spacy/spaCy')
sys.path.insert(0, r'/home/albrecht/spacy/spacy-lookups-data')

for p in sys.path:
    print(p)

In [ ]:
import spacy

nlp = spacy.blank('de')
#nlp = spacy.load('de')

def print_nlp(doc):
    print(" ".join([f"{t}/{t.lemma_}/{t.pos_}" for t in doc if not t.is_punct]))

texts = """Dieser Gärtner wohnt im Haus.
"""

for text in texts.split("\n"):
    doc = nlp(text)
    print_nlp(doc)

## Changes

In [ ]:
import json

path = "/home/albrecht/spacy/spacy-lookups-data/spacy_lookups_data/data/"
with open(path+"de_lemma_lookup.json", "r") as file:
    LOOKUP = json.loads(file.read())

In [ ]:
LOOKUP['Haus']
LOOKUP['Gärtner']

In [ ]:
file = sys.stdout
# file = open("lemmatizer_de.py", "w", encoding='utf-8')
# file = open("lemmatizer_de_changes.py", "w", encoding='utf-8')

new_lookup = {}

for word, lemma in LOOKUP.items():
    try:
        if word[0].isupper() and lemma.islower() and word.lower() not in LOOKUP and word in nouns:
            new_lemma = nouns[word]
            new_lookup[word] = new_lemma
            # file.write(f'    "{word}": "{new_lemma}",\n')
            # file.write(f'    "{word}": "{new_lemma}", # previous "{lemma}"\n')
            print(f'    "{word}": "{new_lemma}" # previous "{lemma}"')
        else:
            # pass
            new_lookup[word] = LOOKUP[word]
        
    except Exception as e:
        print(e)
        print(f'    "{word}": "{lemma}"')
        break
        
new_lookup['Aber'] = 'aber'

In [ ]:
new_lookup['Haus']
new_lookup['Gärtner']

In [ ]:
len(new_lookup)
len(LOOKUP)

In [ ]:
import unicodedata

path = "/home/albrecht/spacy/spacy-lookups-data/spacy_lookups_data/data/"
with open(path+"de_lemma_lookup.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(new_lookup, indent=4, ensure_ascii=False)+"\n")

In [ ]:
new_lookup['Abbröckelnden']

In [ ]:
for i, word in enumerate(new_lookup.keys()):
    print(i, word)
    if i > 200:
        break

In [ ]:
"Banden"[:-1] in LOOKUP

In [ ]:
"Bande" in LOOKUP

In [ ]:
import spacy

In [ ]:
nlp.vocab.lookups.get_table.